In [1]:
from variational_circuit.varicirc import *
from variational_circuit.measure import *
from scipy.optimize import minimize
from time import time

from qutip.random_objects import rand_ket
from qutip.qip.qubits import qubit_states
from qutip.tensor import tensor

In [2]:
N = 4 # number of qubits
n = 2 # number of reduced qubits

# Random inputs (qubit state)
Input = rand_ket(2**N,dims=[[2]*N,[1]*N])
# Reference State
r_state = qubit_states(n)

In [3]:
def test_ptest(x):
    vc1.update_ansatzes(x)
    return 1-qcirc_test(Input,vc1.qc,[[0,1],[2,3]],test_func=sep_purity)

def test_phase2(x):
    vc11.update_ansatzes(x)
    return 1-qcirc_test(state2,vc11.qc,r_state,test_func=fid_ref)

def test_qdr(x):
    vc2.update_ansatzes(x)
    return 1-qcirc_test(Input,vc2.qc,r_state,[2,3],test_func=fid_ref)

def test_entropy(x):
    vc3.update_ansatzes(x)
    return qcirc_test(Input,vc3.qc,[2,3],test_func=c_entropy)

In [4]:
print("disentangling")
vc1 = vcirc(N)

for L in np.arange(1,5):    # number of ansatz
    vc1.add_ansatz(np.zeros(N*3)) # add one layer
    t0 = time()
    x0 = np.zeros(N*3*L) # init parameters

    res = minimize(test_ptest,x0,method='powell')
    print("L=",L,"Result=",res.fun,"Iteration time=",res.nit,"Time=",time()-t0,"seconds")
    if res.fun < 1e-5:
        break
    
print("The second phase")
vc11 = vcirc(n)

prop = gate_sequence_product(vc1.qc.propagators())
out = prop*Input
state2 = out.ptrace([2,3])

for L in np.arange(1,5):
    vc11.add_ansatz(np.zeros(n*3)) # add one layer
    t0 = time()
    x0 = np.zeros(n*3*L) # init parameters

    res = minimize(test_phase2,x0,method='powell')
    print("L=",L,"Result=",res.fun,"Iteration time=",res.nit,"Time=",time()-t0,"seconds")
    if res.fun < 1e-5:
        break

prop2 = gate_sequence_product(vc11.qc.propagators())
out2 = prop2*state2*prop2.dag()
final = tensor(out.ptrace([0,1]),out2)

print("result fidelity is",fidelity(final,tensor(out.ptrace([0,1]),r_state*r_state.dag())))


disentangling
L= 1 Result= 0.6132630034952536 Iteration time= 16 Time= 32.447409868240356 seconds
L= 2 Result= 0.0328622887061375 Iteration time= 38 Time= 261.6378073692322 seconds
L= 3 Result= 4.474198789239381e-14 Iteration time= 46 Time= 858.0398941040039 seconds
The second phase
L= 1 Result= 0.07633017106406759 Iteration time= 5 Time= 2.360589027404785 seconds
L= 2 Result= 6.328271240363392e-15 Iteration time= 24 Time= 47.83451557159424 seconds
result fidelity is 1.0000000419816581


In [5]:
print("original dimensionality reduction")
vc2 = vcirc(N)

for L in np.arange(1,5):    # number of ansatz
    vc2.add_ansatz(np.zeros(N*3)) # add one layer
    t0 = time()
    x0 = np.zeros(N*3*L) # init parameters

    res = minimize(test_qdr,x0,method='powell')
    print("L=",L,"Result=",res.fun,"Iteration time=",res.nit,"Time=",time()-t0,"seconds")
    if res.fun < 1e-5:
        break
prop = gate_sequence_product(vc2.qc.propagators())
out = prop*Input

print("result fidelity is",fidelity(tensor(out.ptrace([0,1]),r_state*r_state.dag()),out))

original dimensionality reduction
L= 1 Result= 0.3220938461609555 Iteration time= 4 Time= 11.603367328643799 seconds
L= 2 Result= 0.12483250391423206 Iteration time= 17 Time= 126.32871270179749 seconds
L= 3 Result= 0.0026711086915088833 Iteration time= 79 Time= 1219.6830115318298 seconds
L= 4 Result= 1.2212453270876722e-15 Iteration time= 50 Time= 1667.0333693027496 seconds
result fidelity is 1.0000000159676705


In [7]:
print("dimensionality reduction with arbitary classical state")
vc3 = vcirc(N)

for L in np.arange(1,5):    # number of ansatz
    vc3.add_ansatz(np.zeros(N*3)) # add one layer
    t0 = time()
    x0 = np.zeros(N*3*L) # init parameters

    res = minimize(test_entropy,x0,method='powell')
    print("L=",L,"Result=",res.fun,"Iteration time=",res.nit,"Time=",time()-t0,"seconds")
    if res.fun < 1e-5:
        break
prop = gate_sequence_product(vc3.qc.propagators())
out = prop*Input
conv_ref = out.ptrace([2,3])
print("The ref state is",conv_ref)
print("result fidelity is",fidelity(tensor(out.ptrace([0,1]),conv_ref),out))

dimensionality reduction with arbitary classical state
L= 1 Result= 1.7324341094770297 Iteration time= 10 Time= 19.77413558959961 seconds
L= 2 Result= 1.1126691747942756 Iteration time= 20 Time= 134.587060213089 seconds
L= 3 Result= 0.05635865332248012 Iteration time= 69 Time= 1160.5224804878235 seconds
L= 4 Result= 2.5873544499925533e-21 Iteration time= 57 Time= 1901.2858355045319 seconds
The ref state is Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 7.88899492e-24+0.00000000e+00j  0.00000000e+00+1.52289276e-24j
   1.31280379e-12+2.81635180e-13j -3.39002734e-24-8.16881258e-24j]
 [ 0.00000000e+00-1.52289276e-24j  3.49745536e-24+0.00000000e+00j
  -2.80321775e-13-1.33613793e-13j -3.66973134e-24+3.38209748e-24j]
 [ 1.31280379e-12-2.81635180e-13j -2.80321775e-13+1.33613793e-13j
   1.00000000e+00+0.00000000e+00j -6.47246563e-13+8.37680479e-13j]
 [-3.39002734e-24+8.16881258e-24j -3.66973134e-24-3.38209748e-24j
  -6.47246563e-13-8.37680479e